## Data
This program is designed to calculate the position of segmented foci relative to individually traced homologus axes (typically asynapsed and/or super-resolution images). 

Users will first need to calculate the proximity of paired axes by running .traces files in the "Paired axis proximity" Jupyter Notebook, or associated .py scripts to produce an "Axis_Trace_Measurements.csv" file. A labelmap image of segmented focal signalis also required, this can be generated using the "Image Segmentation.ijm" ImageJ macro. 

These files must be located in the metadata folder generated by the "Image Segmentation.ijm" segmentation process. The original image should also be present in the folder containing the metadata folders i.e. one directory up. 

## Specify package locations:

Set the paths below to the directories where you saved the TracePy and meiosis_toolkit packages. 

In [1]:
Meio_toolkit_path="Z:\\Meiosis Plugin Paper\Methods in Molecular Biology - Meiosis image analysis\\Code for GitHub\\Python tools\\meiosis_toolkit"

## Add package addresses to the environment

In [2]:
import sys
sys.path.append(Meio_toolkit_path)

## Set directory containing the subfolders of necessary image metadata. These will be batch analysed:

In [3]:
img_library="\\\\cmvm.datastore.ed.ac.uk\\igmm\\iadams-lab\\Meiosis Plugin Paper\\Methods in Molecular Biology - Meiosis image analysis\\Figures\\SIM test data"

## Specify Identities:
Specify the name given to the segmented focal labemap and axial mask you wish to analyse. 
Also specify the channel number containing the focal images in the original stack image.**NB. Use channel numbering as in Fiji i.e. first channel is channel 1. This will be corrected for Python numbering later.**

The .traces file will be automatiaclly selected from the shared metadata folder.

In [4]:
Foci_Labelmap_name="RAD51.tif"#foci labelmap file containing the segmented foci to measure to
Foci_Channel=2
Foci_name="RAD51"#used in output file naming

## Foci included in analysis are filtered by area measured in pixels. Inspect your images to determine reasonable limits for these parameters:

In [5]:
Foci_area_min=5
Foci_area_max=100

## Run:

In [6]:
import os
import pandas as pd
import Shuffling
from skimage import io
from timeit import default_timer
from Focus_position_on_homologous_pairs import Resolved_homologues_focus_position as meio
import fnmatch

#Loop through the folders of image metadata
Folders=next(os.walk(img_library))[1]
for Folder in Folders:
    start = default_timer()
    Folder_Path=img_library+"\\"+Folder+"\\"
    
    #Only process if the necessary files are present in an image metadata folder
    Foci_found=any(Foci_Labelmap_name in file for file in os.listdir(Folder_Path))
    Traces_found=any("Axis_Trace_Measurements.csv" in file for file in os.listdir(Folder_Path))
    if (Foci_found & Traces_found):
        Foci_path=Folder_Path+Foci_Labelmap_name
        Traces_path=Folder_Path+"Axis_Trace_Measurements.csv"
        #Open files
        Foci_Labelmap=io.imread(Foci_path)
        Trace_df=pd.read_csv(Traces_path)
        
        Foci_Channel_Corrected=Foci_Channel-1
        img=Folder.replace("_Output", "")
        Image_Path=img_library+"\\"+img+".tif"
        Foci_Intensity_Img=io.imread(Image_Path)[:,:,Foci_Channel_Corrected]
        
        Foci_Measurements=meio(Trace_df, Foci_Labelmap, Foci_Intensity_Img, Foci_area_min, Foci_area_max)
        
        #Save output
        output_name=Foci_name+"_homologous_axis_measurements.csv"
        Foci_Measurements.to_csv(Folder_Path+"\\"+output_name)
        
        print(Folder+" processed")
        

SIM test image_Output processed
